In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy, scipy as sklearn, librosa, urllib
import librosa.display
from IPython.display import Audio
import json 
import seaborn as sns
from sklearn.cluster import KMeans
import csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score
import keras
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from os import path

from itertools import cycle
from sklearn.manifold import TSNE
from sklearn.metrics import roc_curve, auc, silhouette_score,roc_auc_score, precision_recall_fscore_support
from tqdm import tqdm
import opensmile


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import regularizers
from joblib import dump, load

from tensorflow.keras.datasets import mnist


In [86]:
def compute_dur(duration):
    tot = 0
    for i in range(len(duration)):
        tot += duration[i].microseconds / 1000

    return tot / i, i

In [3]:

not_aug_df=pd.read_pickle('Datasets/EMOVO/not_aug_df.pkl')
# semi_aug_df2=pd.read_pickle('EMOVO_dataset/semi_aug_df.pkl')
# aug_df2=pd.read_pickle('EMOVO_dataset/aug_df.pkl')

In [14]:
scaler = StandardScaler()
#extraction of the mfccs from the datasets - not aug
X_mfccs_k = np.array(not_aug_df.iloc[:, 3:29])
X_mfccs_k=np.array(X_mfccs_k.tolist())
X_mfccs_k=scaler.fit_transform(X_mfccs_k.reshape(-1, X_mfccs_k.shape[-1])).reshape(X_mfccs_k.shape)
Y_mfccs_k=not_aug_df['label']

#extraction of the log mel specrogram from the datasets - not aug
X_logmel_k = np.array(not_aug_df.loc[:, ['logmel' in i for i in not_aug_df.columns]])
X_logmel_k=np.array(X_logmel_k.tolist())
X_logmel_k=scaler.fit_transform(X_logmel_k.reshape(-1, X_logmel_k.shape[-1])).reshape(X_logmel_k.shape)
Y_logmel_k=not_aug_df['label']


#extraction of labels_id from datasets
Y_not_aug=not_aug_df['label_id']
# Y_semi_aug=semi_aug_df['label_id']
# Y_aug=aug_df['label_id']



In [6]:
#take the log mel spectrogram from the datasets
X_logmel = np.array(not_aug_df.loc[:, ['logmel' in i for i in not_aug_df.columns]])
X_logmel=np.array(X_logmel.tolist())
# X_logmel_semi_aug = np.array(semi_aug_df.loc[:, ['logmel' in i for i in semi_aug_df.columns]])
# X_logmel_semi_aug=np.array(X_logmel_semi_aug.tolist())
# X_logmel_aug = np.array(aug_df.loc[:, ['logmel' in i for i in aug_df.columns]])
# X_logmel_aug=np.array(X_logmel_aug.tolist())

In [7]:
#take the mfccs & deltas from the datasets
X_mfccs = np.array(not_aug_df.iloc[:, 3:29]) 
X_mfccs=np.array(X_mfccs.tolist())
# X_mfccs_semi_aug = np.array(semi_aug_df.iloc[:, 3:29])
# X_mfccs_semi_aug=np.array(X_mfccs_semi_aug.tolist())
# X_mfccs_aug = np.array(aug_df.iloc[:, 3:29])
# X_mfccs_aug=np.array(X_mfccs_aug.tolist())

In [8]:
#add one dimension to data to make them suitable foR CNN
X_logmel = np.reshape(X_logmel, (X_logmel.shape[0],X_logmel.shape[1],X_logmel.shape[2],1))
# X_logmel_semi_aug= np.reshape(X_logmel_semi_aug, (X_logmel_semi_aug.shape[0],X_logmel_semi_aug.shape[1],X_logmel_semi_aug.shape[2],1))
# X_logmel_aug= np.reshape(X_logmel_aug, (X_logmel_aug.shape[0],X_logmel_aug.shape[1],X_logmel_aug.shape[2],1))

In [9]:
#CNN creation
def create_cnn(input_data):
  modelcnn = keras.Sequential()


  modelcnn.add(keras.layers.Conv2D(32, (2, 6), activation='relu', input_shape=input_data,kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.MaxPooling2D((2, 7), strides=(2,2),padding='same'))
  modelcnn.add(keras.layers.Dropout(rate=0.3))
  
  modelcnn.add(keras.layers.Conv2D(64, (2, 6), activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.MaxPooling2D((2,7),strides=(2,2), padding='same'))
  modelcnn.add(keras.layers.Dropout(rate=0.3))


  modelcnn.add(keras.layers.Conv2D(128, (2, 6), activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.MaxPooling2D((2, 7),strides=(2,2), padding='same'))
  modelcnn.add(keras.layers.Dropout(rate=0.3))


  modelcnn.add(keras.layers.Conv2D(256, (2, 6), activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.MaxPooling2D((2, 7),strides=(2,2), padding='same'))
  modelcnn.add(keras.layers.Dropout(rate=0.3))

 
  modelcnn.add(keras.layers.GlobalAveragePooling2D())
  modelcnn.add(keras.layers.Dense(128, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.Dropout(0.3))
  modelcnn.add(keras.layers.Dense(256, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.Dropout(0.3))

  modelcnn.add(keras.layers.Dense(7, activation='softmax'))
  
  return modelcnn

In [10]:
input_data=(X_logmel.shape[1:])
modelcnn=create_cnn(input_data)
modelcnn.summary()

c:\Users\anton\Documents\PhD\Speech_Emotion_Recognition\EMOVO_dataset\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 59, 125, 32)    │           416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 30, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 58, 64)     │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 15, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 15, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 14, 24, 128)    │        98,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 7, 12, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 7, 12, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 6, 7, 256)      │       393,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 3, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 3, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 584,679 (2.23 MB)

 Trainable params: 584,679 (2.23 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
    def __init__(self, metric_name = "duration"):
        self.__epoch_start = None
        self.__metric_name = metric_name

    def on_epoch_begin(self, epoch, logs=None):
        self.__epoch_start = datetime.now()

    def on_epoch_end(self, epoch, logs=None):
        logs[self.__metric_name] = datetime.now() - self.__epoch_start

In [68]:
from datetime import datetime
import time
class timecallback(tf.keras.callbacks.Callback):
    def __init__(self, metric_name = "duration"):
        self.__epoch_start = None
        self.__metric_name = metric_name
    def on_epoch_begin(self, epoch, logs=None):
        self.__epoch_start = datetime.now()
    def on_epoch_end(self, epoch, logs=None):
        logs[self.__metric_name] = datetime.now() - self.__epoch_start
    # def on_train_end(self,logs = {}):
    #     plt.xlabel('Epoch')
    #     plt.ylabel('Total time taken until an epoch in seconds')
    #     plt.plot(*zip(*self.times))
    #     plt.show()
    #     from operator import itemgetter
    #     previous_time = 0
    #     for item in self.times:
    #         print("Epoch ", item[0], " run time is: ", item[1]-previous_time)
    #         previous_time = item[1]
    #     print("Total trained time is: ", previous_time)

callbacks = [
timecallback(),
# keras.callbacks.ModelCheckpoint(
#     filepath = name,
#     save_best_only=True,
#     verbose=0,
#     monitor="val_loss"),
keras.callbacks.EarlyStopping(  
    monitor="val_loss",
    min_delta=0.001,
    patience=40,
    verbose=0,
    mode="auto",
    restore_best_weights=True
)

]

In [69]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_logmel, Y_not_aug, test_size=0.10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [84]:

modelcnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

histories = []
for i in range(10):
    histories.append(modelcnn.fit(X_train, y_train, validation_data=(X_val,y_val), batch_size=256,epochs=1000,callbacks=callbacks, verbose=0))


In [85]:
tot_dur = []
tot_epochs = []
for history in histories: 
    durations = history.history["duration"]
    print(f"Mean time: {compute_dur(durations)[0]:.2f}, Total epochs : {compute_dur(durations)[1]}")
    tot_dur.append(compute_dur(durations)[0])
    tot_epochs.append(compute_dur(durations)[1])


print(f"Mean epoch time : {np.mean(tot_dur):.2f}, mean epoch tot : {np.mean(tot_epochs)}")

Mean time: 278.73, Total epochs : 201
Mean time: 302.34, Total epochs : 39
Mean time: 300.09, Total epochs : 40
Mean time: 301.70, Total epochs : 39
Mean time: 301.85, Total epochs : 39
Mean time: 301.65, Total epochs : 39
Mean time: 302.23, Total epochs : 39
Mean time: 302.47, Total epochs : 39
Mean time: 303.04, Total epochs : 39
Mean time: 302.28, Total epochs : 39
Mean epoch time : 299.64, mean epoch tot : 55.3


In [73]:
#create the multilayer perceptron
from keras.layers import ReLU
def create_mlp(): 
  model = keras.Sequential([

    
        keras.layers.Flatten(input_shape=(X_mfccs.shape[1], X_mfccs.shape[2])),
       
        keras.layers.Dense(1024,activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),
      
   
        keras.layers.Dense(512,activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),
       
    
        keras.layers.Dense(256,activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),

       
        keras.layers.Dense(7, activation='softmax')
    ])

  

  model.compile(optimizer="adam",
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

  return model


 

In [74]:
y_test.unique()

array([4, 5, 3, 1, 6, 2, 0])

In [75]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_mfccs, Y_not_aug, test_size=0.10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [76]:

histories = []
for i in range(10):
    model=create_mlp()
    histories.append(model.fit(X_train, y_train, verbose=0,  validation_data=(X_val,y_val),epochs=500,callbacks=callbacks))

In [83]:
tot_dur = []
tot_epochs = []
for history in histories: 
    durations = history.history["duration"]
    print(f"Mean time: {compute_dur(durations)[0]:.2f}, Total epochs : {compute_dur(durations)[1]}")
    tot_dur.append(compute_dur(durations)[0])
    tot_epochs.append(compute_dur(durations)[1])


print(f"Mean epoch time : {np.mean(tot_dur):.2f}, mean epoch tot : {np.mean(tot_epochs)}")

Mean time: 210.83, Total epochs : 39
Mean time: 189.55, Total epochs : 39
Mean time: 190.79, Total epochs : 39
Mean time: 190.01, Total epochs : 39
Mean time: 189.82, Total epochs : 39
Mean time: 192.41, Total epochs : 39
Mean time: 194.46, Total epochs : 39
Mean time: 236.97, Total epochs : 39
Mean time: 245.05, Total epochs : 39
Mean time: 233.56, Total epochs : 39
Mean epoch time : 207.34, mean epoch tot : 39.0


In [60]:
compute_dur(duration)

(189.01071794871797, 39)

In [21]:
data_df = pd.read_csv("Datasets/DEMoS/data.csv")

In [23]:
def return_features(file,pad):
    X, sample_rate = librosa.load(file)
    max_ = X.shape[0] / sample_rate
    # if max_ < pad:
    #     length = (pad * sample_rate) -  (X.shape[0] / sample_rate)
    #     X = librosa.util.pad_center(X, size = length, mode = 'constant')
    smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals,
    )
    y = smile.process_signal(X,sample_rate)
    #  smile.process_file(file, end=3)
    return y
def get_max_min(files):
    min_, max_ = 100, 0
    for file in files:
        sound_file, samplerate = librosa.load(file)
        t = sound_file.shape[0] / samplerate
        if t < min_:
            min_ = t
        if t >= max_:
            max_ = t

    return np.round(max_,2) + 0.01, min_

In [24]:
train_data = pd.DataFrame(columns=['filename', 'features', 'label'])
max, min = get_max_min('Datasets/DEMoS/'+data_df.file_name)
filenames= data_df.file_name
labels= data_df.label

tot = range(list(data_df.index)[-1])



for index,file in tqdm(zip(tot,filenames)):
    train_data.loc[index] = [file, return_features('Datasets/DEMoS/'+file,max), labels[index]]


9696it [10:08, 15.93it/s]


In [33]:
emo = []
tot = 0

for id, name in zip(train_data["filename"],train_data["label"]):
    # if "PR" in id:
        emo.append(id)
        


mask = train_data['filename'].isin(emo)
emo_df = train_data[mask].reset_index()
emo_df = emo_df.drop(['index'], axis=1)



In [34]:
data_classes = (list((train_data["label"].unique())))
Y = keras.utils.to_categorical(list((train_data["label"].apply(data_classes.index))))
X = np.stack(train_data["features"])
_X= (X-X.min())/(X.max()-X.min())
X_train, X_test, y_train, y_test = train_test_split(_X, Y, test_size=0.2, random_state=22)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=22)

# _X_train = X_train.to_numpy()
# _X_val = X_val.to_numpy()
# _X_test = X_test.to_numpy()

In [47]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)


SVC_model = SVC(kernel = 'linear', gamma = 'auto', probability = True, verbose=True)

In [48]:
SVC_history = SVC_model.fit(X_train, np.argmax(y_train,axis=-1))
SVC_model.score(X_test,np.argmax(y_test,axis=-1))   

[LibSVM]

0.1788659793814433

In [49]:
y_pred = SVC_model.predict(X_test)  

print("Accuracy:", accuracy_score(np.argmax(y_test,axis=-1), y_pred))

Accuracy: 0.1788659793814433


In [1]:
import keras

In [18]:
def create_mlp(): 
  model = keras.Sequential([

    
        keras.layers.Flatten(input_shape=(256,256)),
       
        keras.layers.Dense(1024,activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),
      
   
        keras.layers.Dense(512,activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),
       
    
        keras.layers.Dense(256,activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),

       
        keras.layers.Dense(7, activation='softmax')
    ])
  return model

In [2]:
#CNN creation
def create_cnn(input_data):
  modelcnn = keras.Sequential()


  modelcnn.add(keras.layers.Conv2D(32, (2, 6), activation='relu', input_shape=input_data,kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.MaxPooling2D((2, 7), strides=(2,2),padding='same'))
  modelcnn.add(keras.layers.Dropout(rate=0.3))
  
  modelcnn.add(keras.layers.Conv2D(64, (2, 6), activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.MaxPooling2D((2,7),strides=(2,2), padding='same'))
  modelcnn.add(keras.layers.Dropout(rate=0.3))


  modelcnn.add(keras.layers.Conv2D(128, (2, 6), activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.MaxPooling2D((2, 7),strides=(2,2), padding='same'))
  modelcnn.add(keras.layers.Dropout(rate=0.3))


  modelcnn.add(keras.layers.Conv2D(256, (2, 6), activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.MaxPooling2D((2, 7),strides=(2,2), padding='same'))
  modelcnn.add(keras.layers.Dropout(rate=0.3))

 
  modelcnn.add(keras.layers.GlobalAveragePooling2D())
  modelcnn.add(keras.layers.Dense(128, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.Dropout(0.3))
  modelcnn.add(keras.layers.Dense(256, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
  modelcnn.add(keras.layers.Dropout(0.3))

  modelcnn.add(keras.layers.Dense(7, activation='softmax'))
  
  return modelcnn

In [19]:
model = create_mlp()

c:\Users\anton\Documents\PhD\Speech_Emotion_Recognition\EMOVO_dataset\.venv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [23]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_3 (Flatten)             │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1024)           │    67,109,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 67,767,815 (258.51 MB)

 Trainable params: 67,767,815 (258.51 MB)

 Non-trainable params: 0 (0.00 B)

In [1]:
from audiomentations import Compose, Gain, PitchShift, Resample
import numpy as np


In [5]:
"""Name, Pitch, Downsample, Volume boost


DEFAULT_PRESETS = [
    Preset("Man", -1.5, None, None),
    Preset("Woman", 2.5, None, None),
    Preset("Boy", 1.25, None, None),
    Preset("Girl", 2.8, None, None),
    Preset("Darth Vader", -6.0, None, None),
    Preset("Chipmunk", 10.0, None, None),
    Preset("Bad Mic", None, 8, 0),
    Preset("Radio", None, 6, 0),
    Preset("Megaphone", None, 2, 0),
    Preset("Off", 0.0, None, None)
]

"""

'Name, Pitch, Downsample, Volume boost\n\n\nDEFAULT_PRESETS = [\n    Preset("Man", -1.5, None, None),\n    Preset("Woman", 2.5, None, None),\n    Preset("Boy", 1.25, None, None),\n    Preset("Girl", 2.8, None, None),\n    Preset("Darth Vader", -6.0, None, None),\n    Preset("Chipmunk", 10.0, None, None),\n    Preset("Bad Mic", None, 8, 0),\n    Preset("Radio", None, 6, 0),\n    Preset("Megaphone", None, 2, 0),\n    Preset("Off", 0.0, None, None)\n]\n\n'

In [ ]:
def preset(pitch,volume,resample):

    
    aug = Compose([
        PitchShift(min_semitones=pitch, max_semitones=pitch, p=p1),
        Resample(min_sample_rate = resample, max_sample_rate= resample, p=p2),
        Gain(min_gain_db = volume, max_gain_db = volume, p=p3)
    ])
    return aug

In [2]:
man = Compose([
    PitchShift(min_semitones=-1.5, max_semitones=-1.5, p=1),
])


woman = Compose([
    PitchShift(min_semitones=2.5, max_semitones=2.5, p=1),
])


boy = Compose([
    PitchShift(min_semitones=1.25, max_semitones=1.25, p=1),
    Gain(min_gain_db = 0, max_gain_db=0, p=0),
    Resample(min_sample_rate=0, max_sample_rate=0, p=0)
])



girl = Compose([
    PitchShift(min_semitones=2.8, max_semitones=2.8, p=1),
    Gain(min_gain_db = 0, max_gain_db=0, p=0),
    Resample(min_sample_rate=0, max_sample_rate=0, p=0)
])

In [3]:
import pandas as pd


data_df = pd.read_csv("EMOVO_Dataset/data.csv")

In [4]:
import librosa
from IPython.display import Audio

y,sr=librosa.load('EMOVO_dataset/f1/tri-f1-l1.wav')

Audio(y, rate=sr)


In [5]:
Audio(man(y,sr), rate=sr)